In [1]:
!pip install datasets transformers soundfile librosa editdistance torch torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
from pathlib import Path
import librosa
import soundfile as sf
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import editdistance

In [3]:
RESAMPLED_AUDIO_DIR = "resampled_audio"
os.makedirs(RESAMPLED_AUDIO_DIR, exist_ok=True)

In [4]:
def resample_audio(input_path, output_path, target_sr=16000):
    audio, sr = librosa.load(input_path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(y=audio, orig_sr=sr, target_sr=target_sr)
    sf.write(output_path, audio, target_sr)
    return output_path

In [5]:
cv_train = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="train")
cv_test = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test")
cv_valid = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="validation")

combined_data = {'path': [], 'sentence': []}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...


Reading metadata...: 1816it [00:00, 114732.42it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2819it [00:00, 136882.15it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2237it [00:00, 111607.96it/s]


In [6]:
for dataset in [cv_train, cv_test, cv_valid]:
    for item in dataset:
        original_path = item['audio']['path']
        filename = Path(original_path).name
        resampled_path = os.path.join(RESAMPLED_AUDIO_DIR, filename)

        if not os.path.exists(resampled_path):
            resampled_path = resample_audio(original_path, resampled_path)

        combined_data['path'].append(resampled_path)
        combined_data['sentence'].append(item['sentence'])

In [7]:
train_paths, temp_paths, train_sentences, temp_sentences = train_test_split(
    combined_data['path'], combined_data['sentence'],
    test_size=0.2, random_state=64
)

valid_paths, test_paths, valid_sentences, test_sentences = train_test_split(
    temp_paths, temp_sentences,
    test_size=0.5, random_state=64
)

In [8]:
os.makedirs("cv_data/cv_test", exist_ok=True)
with open("cv_data/cv_test/wav.scp", "w") as wav_file, \
     open("cv_data/cv_test/text", "w") as text_file, \
     open("cv_data/cv_test/utt2spk", "w") as utt2spk_file:

    for idx, (path, text) in enumerate(zip(test_paths, test_sentences)):
        uttid = f"utterance_{idx}"
        wav_file.write(f"{uttid} {path}\n")
        text_file.write(f"{uttid} {text}\n")
        utt2spk_file.write(f"{uttid} global\n")

In [9]:
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [23]:
def whisper_transcribe(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)
    inputs = processor(audio_input, return_tensors="pt", sampling_rate=16000)
    model.to('cuda')
    inputs = {key: value.cuda() for key, value in inputs.items()}
    attention_mask = (inputs['input_features'] != 0).float()

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features=inputs['input_features'],
            attention_mask=attention_mask,
            language='mr'
        )

    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription


In [24]:
predictions = []
for idx, path in enumerate(test_paths):
    transcription = whisper_transcribe(path)
    predictions.append(transcription)

In [25]:
with open("cv_test_results.txt", "w") as f:
    for idx, transcription in enumerate(predictions):
        f.write(f"utterance_{idx} {transcription}\n")

In [26]:
def calculate_error_rates(ref_file, hyp_file):
    with open(ref_file) as f1, open(hyp_file) as f2:
        refs = [line.strip().split(' ', 1)[1] for line in f1]
        hyps = [line.strip().split(' ', 1)[1] for line in f2]

        total_wer = 0
        total_cer = 0

        for ref, hyp in zip(refs, hyps):
            ref_words = ref.split()
            hyp_words = hyp.split()
            word_edits = editdistance.eval(ref_words, hyp_words)
            wer = word_edits / len(ref_words) if ref_words else 1
            total_wer += wer

            char_edits = editdistance.eval(ref, hyp)
            cer = char_edits / len(ref) if ref else 1
            total_cer += cer

        avg_wer = (total_wer / len(refs)) * 100
        avg_cer = (total_cer / len(refs)) * 100

        return avg_wer, avg_cer

In [27]:
wer, cer = calculate_error_rates(
    'cv_data/cv_test/text',
    'cv_test_results.txt'
)

print(f"WER: {wer:.2f}%")
print(f"CER: {cer:.2f}%")

with open("cv_test_results.txt", "a") as f:
    f.write(f"WER: {wer:.2f}%\n")
    f.write(f"CER: {cer:.2f}%\n")

WER: 178.28%
CER: 152.06%
